In [1]:
import math
import numpy as np
import pandas as pd
import sys
import re
from functions import *
from mining_functions import *
from collections import Counter
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy import stats
np.set_printoptions(precision=2)
pd.set_option('precision', 2)
%matplotlib inline
matplotlib.style.use('ggplot')

In [2]:
PATH = '/Google Drive/Jonathan Sarah Ido folder/data/CCK/'
def get_path(path = PATH):
    if os.name == 'posix':
        path = '/Documents/code/Phet-log-analyzer/cck/raw_data_parsing_check/'
        return os.environ['HOME']+path #'/Google Drive/Jonathan Sarah Ido folder/data/CCK/'
    elif os.name == 'nt':
        if os.getenv("COMPUTERNAME") == 'PYRRHA':
                path = '/Documents/git/Phet-log-analyzer/cck/raw_data_parsing_check/'
        else:
            path = '/git/Phet-log-analyzer/cck/raw_data_parsing_check/'
        return os.environ['USERPROFILE']+ path.replace('/','\\') #'\\Google Drive\Jonathan Sarah Ido folder\data\CCK\\'
    else:
        raise Exception('OS not recongnized. I\'m confused.')
        
df = pd.read_csv(get_path() + 'phet_cck_user_actions+sophistication_WITHPAUSE_more_circuit_info.txt',index_col=False)
# dfx = pd.read_csv('C:\Users\Sarah\Documents\git\Phet-log-analyzer\cck\\raw_data_parsing_check\phet_cck_user_actions+sophistication_WITHPAUSE_more_circuit_info.txt',index_col=False)
df["student"] = df["student"].astype('category')
df["Family"]=df["Family"].str.capitalize()
df["Family_tool"]=df["Family_tool"].str.capitalize()
df["Family_default"]=df["Family_default"].str.capitalize()
df["Family_both"]=df["Family_both"].str.capitalize()

df_scores = pd.read_csv(data_path + 'MATCHING_phet_cck_user_data_anonymized.txt')
df_scores["student"] = df_scores["student"].astype('category')

IOError: File C:\Users\Sarah\git\Phet-log-analyzer\cck\raw_data_parsing_check\phet_cck_user_actions+sophistication_WITHPAUSE_more_circuit_info.txt does not exist

In [ ]:
max_time = 25*60*1000
max_times = {s:min(df[df['student']==s]["Time Stamp"])+max_time for s in set(df['student'])}

def keep_by_time (row):
    if row["Time Stamp"] <= max_times[row["student"]]:
        return True
    else: 
        return False

df['keep'] = df.apply (lambda row: keep_by_time (row),axis=1)
df=df[df['keep']]

### Functions used to calculate information gain, plot use, etc...

In [ ]:
def add_text(attribute,family_category,cut_off, shortest_seq_length, longest_seq_length,B):
    text = """Showing sequences for students split by {0}, using the categories {1}.
            Removed sequences used by less than {2}% of students.
            Found sequences of lenght {3} to {4}.
            Using {5} time bins""".format(attribute,family_category,int(cut_off*100), shortest_seq_length, longest_seq_length,B)
    fig = plt.figure(figsize=(10, 4))
    ax = plt.gca()
    ax.text(0.5,0.5,text,
        horizontalalignment='center',
        verticalalignment='center',
        fontsize = 15)
    plt.axis('off')
    return fig

In [ ]:
def rank_sequences(sequence_counts,B,axesnum=None):
    ranks = []
    for seq,counts in sequence_counts.iteritems():
#         if np.sum(counts)>0:
        ranks.append((seq,calc_infogain(counts,B,axesnum)))
    return sorted(ranks, key=lambda tup: tup[1])

def get_top_seqs(ranks,N):
    return ranks[-N:]

### Let's investigate how best to split students

In [ ]:
# df_scores.hist(column='z pre')
# print list(df_scores['z pre'])
sns.set_style("darkgrid")
sns.set_context("paper")
fig, ax = plt.subplots(1, figsize=(5.5,2.5))
sns.distplot(list([x for x in df_scores['z pre'] if x <= 0.45]), 
             bins=7, kde=False, rug=False, color='#3f007d', hist_kws={'alpha':0.7}, label = 'low incoming knowledge');
X = list([x for x in df_scores['z pre'] if x > 0.45])
X.extend([2.45,2.45,2.45])
sns.distplot(X,bins=3, kde=False, rug=False, color='#525252', hist_kws={'alpha':0.7},  label = 'high incoming knowledge');
ax.set_ylabel('Number of students')
ax.set_xlabel('pre-test z-score')
handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles, labels, loc='upper right', bbox_to_anchor=(1.0,0.95))
fig.savefig('figs/hist_pre.svg')

#### Clearly, students' pre test is a bi modal distribution. Let's split them this way.

In [ ]:
median_learning0 = 0.45
df_scores['split pre'] = df_scores.apply (lambda row: label_learning (median_learning0,row,"z pre"),axis=1)
median_learning2 = np.median(df_scores[df_scores['split pre']=='low']['z post t2'])
df_scores['split post t2'] = df_scores.apply (lambda row: label_learning (median_learning2,row,"z post t2"),axis=1)

In [ ]:
for x in ['high','low']:
    for y in ['high','low']:
        print x,y, len(set(df_scores[(df_scores['split pre']==x)&(df_scores['split post t2']==y)]['student']))

In [ ]:
# #making student ids groups csv file for Lauren
# for x in ['high','low']:
#     for y in ['high','low']:
#         for z in list(set(df_scores[(df_scores['split pre']==x)&(df_scores['split post t2']==y)]['student'])):
#             print str(z)+','+x+'-'+y
# #         print x,y, len(set(df_scores[(df_scores['split pre']==x)&(df_scores['split post t2']==y)]['student']))

In [ ]:
for x in ['high','low']:
    for y in ['scaff','not']:
        print x,y, len(set(df_scores[(df_scores['scaffolding']==y)&(df_scores['split pre']==x)]['student']))

In [ ]:
for x in ['high','low']:
    for y in ['scaff','not']:
        print x,y, len(set(df_scores[(df_scores['scaffolding']==y)&(df_scores[df_scores['split pre']=='low']['split post t2']==x)]['student']))

In [ ]:
a = set(get_students('split post t2','high'))
b = set(get_students('used this circuit sim before?',1))
c = set(get_students('split post t2','low'))
print len(a), len(b), len(a.intersection(b)), len(c.intersection(b))

In [ ]:
z = df_scores[df_scores['split pre']=='low']['z pre']
y = df_scores[df_scores['split pre']=='high']['z pre']
print np.mean(z), np.std(z)
print np.mean(y), np.std(y)

In [ ]:
median_learning2 = np.median(df_scores[df_scores['split pre']=='low']['z post t2'])
fig, ax = plt.subplots(1, figsize=(5.5,2.5))
sns.axes_style({'legend.numpoints': 20})
ax.set_ylim(0,13)
bar1 = sns.distplot(list([x for x in df_scores[df_scores['split pre']=='low']['z post t2'] if x <= median_learning2]), 
             bins=5, kde=False, rug=False, color='#3f007d', hist_kws={'alpha':0.7},  label = 'Low-Low learners');
bar2 = sns.distplot(list([x for x in df_scores[df_scores['split pre']=='low']['z post t2'] if x > median_learning2]), 
             bins=6, kde=False, rug=False, color='#3f007d', hist_kws={'alpha':0.7}, label = 'Low-High learners');
# Define some hatches
hatches = ['-', '+', 'x', '\\', '*', 'o','-', '+', 'x', '\\', '*', 'o']
# Loop over the bars
for i,thisbar in enumerate(bar1.patches):
    if i <=4:
    # Set a different hatch for each bar
        thisbar.set_hatch(hatches[5])
    else:
        thisbar.set_hatch(hatches[2])
ax.set_xlabel('post-test z-score')
handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles, labels, loc='upper right', bbox_to_anchor=(.95,0.95))
fig.savefig('figs/hist_post.svg')

In [ ]:
import scipy.stats as st
print st.pearsonr(df_scores[df_scores['split pre']=='low']["z pre"],df_scores[df_scores['split pre']=='low']["z post t2"] )
print st.spearmanr(df_scores[df_scores['split pre']=='low']["z pre"],df_scores[df_scores['split pre']=='low']["z post t2"] )
plt.plot(df_scores[df_scores['split pre']=='low']["z pre"],df_scores[df_scores['split pre']=='low']["z post t2"],'.')

In [ ]:
#making student ids groups csv file for Lauren
for x in ['high','low']:
        print x, 'm', np.mean(list(df_scores[(df_scores['split pre']==x)]['pre']))
        print x,'s', np.std(list(df_scores[(df_scores['split pre']==x)]['post t2']))

In [ ]:
pre_same = list(df_scores['pre'])
post_same = list(df_scores['post identical to pre'])
print len(pre_same)
print np.mean(pre_same),np.mean(post_same)
print np.std(pre_same),np.std(post_same)
t,p =  stats.ttest_ind(post_same,pre_same,equal_var=False)
print t,p
d = (np.mean(post_same)-np.mean(pre_same))/math.sqrt((np.std(pre_same)**2+np.std(post_same)**2)/2)
print d

### We can calculate the information gain of each sequence by time bin 
given a split by pre test

In [ ]:
### if redoing this analysis, fix algorithm wot use frequency of use in each time bin because we have unequal groups

# students = get_students()
# CUT_OFF = 0.35 #we keep only sequences used once by at least 20% of students
# shortest_seq_length = 2
# longest_seq_length = 10
# BINS = 4  #number of bins

# ### PARAMETERS
# attributes = [('split pre','high','low')]
# family_categories = ["Family","Family_tool","Family_default"]

# parse_axis = {0:'time',1:'group',None:'time and group'}
# pdf = PdfPages('infogain_results_by_pre.pdf')

# for attribute,level1,level2 in attributes:
#     for family_category in family_categories:
#         for axis in [0,1,None]:
#             if axis == 1: B = 1
#             else: B = BINS
        
#             print "For attribute {0}, category {1}".format(attribute,family_category)
#             pdf.savefig(add_text(attribute,family_category,CUT_OFF, shortest_seq_length, longest_seq_length,B))
#             sequence_counts = get_sequence_use_by_timebin(df,students,family_category,
#                                                           B,attribute,level1,level2,
#                                                           shortest_seq_length,longest_seq_length,CUT_OFF)
#             print len(sequence_counts)
#             ylabels = [level1,level2]
#             tops = get_top_seqs(rank_sequences(sequence_counts,B,axis),5)
#             for seq,infogain in tops:
# #                 print seq, sequence_counts[seq]
#                 title = '{0}: infogain {1} by {2}'.format(seq,round(infogain,3),parse_axis[axis])
#                 plot = plot_heat_map(sequence_counts[seq],title, ylabels)
#                 pdf.savefig( plot )
#                 plot.clf()
# pdf.close()

### We can calculate the information gain of each sequence by time bin 
given a split by post test, for only students that did poorly on pre test

In [ ]:
students = get_students(attribute='split pre',level='low')
CUT_OFF = 0.35 #we keep only sequences used once by at least 10% of students
shortest_seq_length = 2
longest_seq_length = 10
BINS = 4  #number of bins

### PARAMETERS
attributes = [('split post t2','high','low')]
family_categories = ["Family"]#,"Family_tool","Family_default"]

parse_axis = {0:'time',1:'group',None:'time and group'}
pdf = PdfPages('infogain_results_by_post2.pdf')

for attribute,level1,level2 in attributes:
    for family_category in family_categories:
        for axis in [0,1,None]:
            if axis == 1: B = 1
            else: B = BINS
        
            print "For attribute {0}, categories {1}".format(attribute,family_category)
            pdf.savefig(add_text(attribute,family_category,CUT_OFF, shortest_seq_length, longest_seq_length,B))
            sequence_counts = get_sequence_use_by_timebin(df,students,family_category,
                                                          B,attribute,level1,level2,
                                                          shortest_seq_length,longest_seq_length,CUT_OFF)
            ylabels = [level1,level2]
            tops = get_top_seqs(rank_sequences(sequence_counts,B,axis),15)
#             print parse_axis[axis]
            for seq,infogain in tops:
#                 print seq, infogain
                title = '{0}: infogain {1} by {2}'.format(seq,round(infogain,3),parse_axis[axis])
                plot = plot_heat_map(sequence_counts[seq],title, ylabels)
                pdf.savefig( plot )
                plot.clf()
pdf.close()

### We repeat the same analysis, only remove the first time bin before to see if what students do in other time bins differentiates them as well (though not as significantly).

In [ ]:
# parse_axis = {0:'time',1:'group',None:'time and group'}

# pdf = PdfPages('infogain_results_no_first_bin.pdf')

# for attribute,level1,level2 in attributes:
#     for family_category in family_categories:
#         for axis in [0,1,None]:
#             if axis == 1:
#                 B = 1
#             else:
#                 B = 3
        
#             print "For attribute {0}, categories {1}".format(attribute,family_category)
#             pdf.savefig(add_text(attribute,family_category,N, shortest_seq_length, longest_seq_length,B))
#             sequence_counts = get_sequence_use_by_timebin(df,students,family_category,B,attribute,level1,level2,shortest_seq_length,longest_seq_length,N)
#             sequence_counts = remove_first_bin(sequence_counts)
#             ylabels = [level1,level2]
#             tops = get_top_seqs(rank_sequences(sequence_counts,B,axis),1)
#             for seq,infogain in tops:
#                 print seq, sequence_counts[seq]
#                 title = '{0}: infogain {1} by {2}'.format(seq,round(infogain,3),parse_axis[axis])
#                 plot = plot_heat_map(sequence_counts[seq],title, ylabels)
#                 pdf.savefig( plot )
#                 plot.clf()
# pdf.close()

In [ ]:
# def remove_first_bin(sequence_counts):
#     new_seq_counts = {}
#     for s,c in sequence_counts.iteritems():
#         new_seq_counts[s] = c[:,1:]
#     return new_seq_counts  

In [ ]:
students = get_students(attribute='split pre',level='low')
CUT_OFF = 0.35 #we keep only sequences used once by at least 35% of students
shortest_seq_length = 2
longest_seq_length = 10
B = 1  #number of bins

### PARAMETERS
attribute, level1, level2 = ('split post t2','high','low')
family_category = "Family"
sequence_counts = get_sequence_use_by_timebin(df,students,family_category,
                                                          B,attribute,level1,level2,
                                                          shortest_seq_length,longest_seq_length,CUT_OFF)
print len(set(sequence_counts.keys()))

In [ ]:
tops = get_top_seqs(rank_sequences(sequence_counts,1,1),10)
print tops
tops.sort(key=lambda tup: tup[1],reverse=True)
tops.append(('CPTb',1.275))
tops.append(('CTbCP',1.073))

In [ ]:
sequence_counts

In [ ]:
from tabulate import tabulate
table = [['rank','seq','infogain','lh','ll','chi2','p','chi2 new','p new','fisher\'s p']]
for i,(seq,infogain) in enumerate(tops):
    i = i+1
    lh = int(sequence_counts[seq][0])
    ll = int(sequence_counts[seq][1])
    nlh = 38-lh
    nll = 36-ll
    obs = [[lh,ll],[nlh,nll]]
    
    total = float(lh + nlh + ll + nll)
    result = stats.chisquare(f_obs= [lh,ll],   # Array of observed counts
                    f_exp= [(lh+nlh)*(lh+ll)/total,(ll+nll)*(lh+ll)/total])   # Array of expected counts
    c2 = result.statistic
    p = result.pvalue
    C2,P,_,_ =  stats.chi2_contingency(obs, correction=False)
    _,fp = stats.fisher_exact(obs)
    table.append([str(x) for x in [i,seq, round(infogain,3),lh,ll,round(c2,3),round(p,3),round(C2,3),round(P,3),round(fp,3)]])
print tabulate(table)

In [ ]:
students = get_students(attribute='split pre',level='low')
CUT_OFF = 0.35 #we keep only sequences used once by at least 35% of students
shortest_seq_length = 2
longest_seq_length = 10
B = 4  #number of bins

### PARAMETERS
attribute, level1, level2 = ('split post t2','high','low')
family_category = "Family"
sequence_counts = get_sequence_use_by_timebin(df,students,family_category,
                                                          B,attribute,level1,level2,
                                                          shortest_seq_length,longest_seq_length,CUT_OFF)
print len(set(sequence_counts.keys()))

In [ ]:
chi2 = {'PTsPC':5.580,'CTsPTsP':3.138,'CPCTsC':4.004,'TsPTsPC':3.138,
        'PCTcCTcC':4.004,'PTsP':5.395,'PCPCTcC':3.275,'TcCPC':4.408,
        'TsCTsC':2.973,'CPTs':2.494,'CPTb':None,'CTbCP':None}
numbers = {'PTsPC':(20,7),  'CTsPTsP':(13,5),  'CPCTsC':(5,13), 
           'TsPTsPC':(13,5),  'PCTcCTcC':(5,13),  'PTsP':(23,9),  'PCPCTcC':(5,12), 
           'TcCPC':(8,18),  'TsCTsC':(6,13),  'CPTs':(15,7), 'CPTb':(14,10),'CTbCP':(13,9)}

In [ ]:
sns.set_style("darkgrid")
sns.set_context("paper")
all_seqs = zip(*tops)[0]
fig, axes = plt.subplots(len(all_seqs),1)#, sharex=True, sharey=True)
maximum = int(max([np.amax(counts) for seq,counts in sequence_counts.iteritems() if seq in all_seqs]))
for i,(seq,ig) in enumerate(tops):
    ax = axes[i]
    data = sequence_counts[seq]
    chi = chi2[seq]
    hl,ll = numbers[seq]
    seq = "$"+seq.replace('Ts','T_2').replace('Tb','T_1').replace('Tc','T_m')+"$"
    ax.text(0,2.05, seq, fontsize=14)
    if chi:
        ax.text(1.35,2.05, "$IG$= {0}, $LL$: {1}, $LH$: {2}, $\chi^2$= {3}".format(str(round(ig,2)),ll,hl,round(chi,1)),
                fontsize=10)
    else:
        ax.text(2.0,2.05, "$IG$= {0}, $LL$: {1}, $LH$: {2}".format(str(round(ig,2)),ll,hl), fontsize=10)

    # heatmap = ax.pcolor(data, cmap=plt.cm.Blues, alpha=0.8, vmin=0)
    heatmap = sns.heatmap(data, ax=ax, cmap=plt.cm.Blues,alpha=0.8, vmin=0, vmax=maximum, cbar=False,
                          annot=False) #, annot_kws={'fontweight':'bold'})

    if i==0:
        xlabels = map(str, np.arange(data.shape[1])+1) 
        xlabels = ['1st time bin','2nd','3rd','4th']
        ax.set_xticklabels([],)
        ax.set_xticks(np.arange(data.shape[1]) + 0.5)
        ax.set_xticklabels(xlabels)
        ax.set_yticks(np.arange(data.shape[0]) + 0.5)
        ylabels = ['Low-Low','Low-High']
        ax.set_yticklabels(ylabels)

#         cbar = ax.figure.colorbar(ax.collections[0])
#         cbar.set_ticks([0,1,3,5,7,9,11,13])
#         cbar.set_ticklabels([0,1,3,5,7,9,11,13])
    else: 
        ax.set_xticklabels(['','','',''])
        ax.set_yticklabels(['',''])

        

#     cbar = ax.figure.colorbar(ax.collections[0])
#     cbar.set_ticks([0,1,3,5,7,9,11,13])
#     cbar.set_ticklabels([0,1,3,5,7,9,11,13])

    fig.set_size_inches(4,1.6*len(all_seqs))
    ax.set_frame_on(False)
#     ax.tick_params(
#         axis='both',        # changes apply to both the x and y-axis
#         which='both',       # both major and minor ticks are affected
#         bottom='off',       # ticks along the those edges are off
#         right='off', 
#         left='off',
#         top='off') 
fig.savefig('figs/heatmap_all.svg')